In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
os.chdir(os.path.join(os.getcwd(), 'drive', 'My Drive', 'Colab Notebooks'))
path = os.getcwd()
path_data = os.path.join(path, 'data')


In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as imgplt
import numpy as np
from tensorflow import keras


In [ ]:
batch_size = 16

In [ ]:
df = pd.read_csv('pokemon.csv')
df = df.sample(frac=1, random_state=42)
test_split = 0.8
df_train = df.iloc[:int(len(df) * test_split)]
df_test = df.iloc[int(len(df) * test_split):]
df_train.head()


In [ ]:
file = df_train['file'].iloc[100]
img = imgplt.imread(os.path.join(path_data, file))
plt.imshow(img)
plt.show()


In [ ]:
labels = list(df.columns.values)
labels.remove('file')


In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(samplewise_center=True,
                                                         samplewise_std_normalization=True)


In [ ]:
generator_train = generator.flow_from_dataframe(dataframe=df_train,
                                               directory="data",
                                               x_col='file',
                                               y_col=labels,
                                               class_mode='raw',
                                               batch_size=batch_size,
                                               shuffle=False)


In [ ]:
generator_test = generator.flow_from_dataframe(dataframe=df_test,
                                               directory="data",
                                               x_col='file',
                                               y_col=labels,
                                               class_mode='raw',
                                               batch_size=batch_size,
                                               shuffle=False)


In [ ]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(generator_train.labels, axis=0))
plt.title("Proporção de imagens em cada classe")
plt.show()


In [ ]:
# Crie aqui o seu modelo
# Para compatibilidade do código das próximas células, o modelo deve ser salvo
# em uma variável chamada 'model'
# INÍCIO DO CÓDIGO
img_shape=(256,256,3)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128,3,input_shape=(img_shape),activation='relu'))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128,3,strides=(2,2),activation='relu'))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,3,strides=(2,2),activation='relu'))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(150,activation='softmax'))

model.summary()

In [ ]:
# FIM DO CÓDIGO

model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics='accuracy')
model.summary()

hist = model.fit_generator(generator_train,epochs=10)

eval = model.evaluate(generator_test)

